In [2]:
from ctd.comparison.analysis.tt.tt import Analysis_TT

In [4]:
tt_3bff_path = "/home/csverst/Github/CtDBenchmark/content/trained_models/task-trained/20240503_Fig1_NBFF_NoisyGRU/max_epochs=500 n_samples=1000 latent_size=64 seed=0 learning_rate=0.001/"
tt_MultiTask_path = "/home/csverst/Github/CtDBenchmark/content/trained_models/task-trained/20240513_MultiTask_NoisyGRU_Final/max_epochs=500 n_samples=2000 batch_size=2000 latent_size=128 seed=0 learning_rate=0.001/"
tt_RandomTarget_path = "/home/csverst/Github/CtDBenchmark/content/trained_models/task-trained/20240605_RandomTarget_NoisyGRU_GoStep_ModL2_Delay/max_epochs=2000 latent_size=128 l2_wt=5e-05 proprioception_delay=0.02 vision_delay=0.05 n_samples=1100 n_samples=1100 seed=0 learning_rate=0.001/"

tt_3bff = Analysis_TT(run_name = "tt_3bff", filepath = tt_3bff_path)
# tt_MultiTask = Analysis_TT(run_name = "tt_MultiTask", filepath = tt_MultiTask_path)
tt_RandomTarget = Analysis_TT(run_name = "tt_RandomTarget", filepath = tt_RandomTarget_path)


In [6]:
tt_3bff.simulate_neural_data(subfolder = "", dataset_path="")
# tt_MultiTask.simulate_neural_data()
tt_RandomTarget.simulate_neural_data(subfolder = "", dataset_path="")




In [13]:
from ctd.task_modeling.simulator.neural_simulator import NeuralDataSimulatorGeneral
sim_3bff=  tt_3bff.simulator

embed_dict = {
    "rect_func": 'exp', # exp, sigmoid, softplus
    "fr_scaling": 2.0,
    
}

noise_dict_poisson = {
    "obs_noise": "pseudoPoisson",
    "dispersion": 1.0
}
sim_3bff_new = NeuralDataSimulatorGeneral(embed_dict=embed_dict, noise_dict=noise_dict_poisson)

tt_3bff.simulator = sim_3bff_new
tt_3bff.simulate_neural_data(subfolder = "poisson", dataset_path="")

import pickle as pkl
with open("tt_3bff.pkl", "wb") as f:
    pkl.dump(tt_3bff, f)

model = tt_3bff.wrapper
with open("model.pkl", "wb") as f:
    pkl.dump(model, f)

datamodule = tt_3bff.datamodule
with open("datamodule_sim.pkl", "wb") as f:
    pkl.dump(datamodule, f)
with open("simulator.pkl", "wb") as f:
    pkl.dump(sim_3bff_new, f)

In [18]:
import os
from ctd.comparison.analysis.dt.dt import Analysis_DT
from ctd.comparison.comparison import Comparison

comparison = Comparison(comparison_tag ="resim_test")
resim_path = "/home/csverst/Github/CtDBenchmark/content/trained_models/task-trained/tt_3bff/n_neurons_50_obs_noise_pseudoPoisson_dispersion_1.0_see/"

path_GRU_Sweep = resim_path + "20240606_GRU_Resim3BFF/"
subfolders_GRU = [f.path for f in os.scandir(path_GRU_Sweep) if f.is_dir()]
tt_3bff_resim = Analysis_TT(run_name = "tt_3bff_resim", filepath = resim_path)
comparison.load_analysis(tt_3bff_resim, group = "tt_3bff_resim", reference_analysis=True)
for subfolder in subfolders_GRU:
    subfolder = subfolder + "/"
    analysis_GRU = Analysis_DT(run_name = "GRU", filepath = subfolder, model_type = "SAE")
    comparison.load_analysis(analysis_GRU, group = "GRU")



In [19]:
comparison.compare_rate_state_r2()

ValueError: No reference index provided